In [77]:
import librosa
import re
import os
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn import svm
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import roc_curve

In [52]:
# settings
dataPath = ".\\wav"

In [60]:
# find personId from the file path
def getPerson(filePath):
    matchObj = re.match(r'.*id(\d+).*', filePath)
    return int(matchObj.group(1))

# sample class
class VoiceSample:
    path = ''
    personId = -1
    feature = []
    
    def __init__(self, path):
        self.path = path
        self.personId = getPerson(path)
        self.getAvgMFCC()
        
    def __str__(self):
        return self.path
    
    def getAvgMFCC(self):
        y, sr = librosa.load(self.path,sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=13)
        self.feature = np.average(mfccs, 1)

In [61]:
def gci(filepath, res):
    files = os.listdir(filepath)
    for fi in tqdm(files):
        fi_d = os.path.join(filepath,fi)
        if os.path.isdir(fi_d):
            gci(fi_d, res)                  
        else:
            voice = VoiceSample(fi_d)
            res[0].append(voice)
            res[1].append(voice.personId)
    
def getList(filepath):
    res = [[],[]]
    gci(filepath, res)
    return res

#递归遍历/root目录下所有文件
data = getList(dataPath)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(data[0], data[1], test_size=0.33, random_state=42)

In [69]:
def extractX(X):
    res = []
    for voice in X:
        res.append(voice.feature)
    return res

in_X_train = extractX(X_train)

In [99]:
gamma = [0.0005, 0.0007, 0.0009, 0.0011, 0.0013]

In [100]:
in_X_test = extractX(X_test)
for g in gamma:
    clf =svm.SVC(kernel = "rbf", gamma=g)
    clf.fit(in_X_train, y_train) 
    y_pred = clf.predict(in_X_test)
    accuracy = accuracy_score(y_test,y_pred)
    print g, accuracy

0.0005 0.7110006215040398
0.0007 0.7333747669359851
0.0009 0.7389683032939715
0.0011 0.7458048477315102
0.0013 0.7445618396519578


In [115]:
y_test = np.asarray(y_test).astype(int)
y_test.shape

(1609L,)

In [104]:
len(in_X_test)

1609

In [107]:
clf =svm.SVC(kernel = "rbf", gamma=0.001)
clf.fit(in_X_train, y_train)
y_score = clf.decision_function(in_X_test)
fpr, tpr, thresholds = roc_curve(np.asarray(y_test), y_score, pos_label = 1)

plt.figure(figsize = (10, 10))
plt.plot(fpr, tpr, linewidth=2)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate', fontsize = 20)
plt.ylabel('True Positive Rate', fontsize = 20)
plt.title('ROC Curve', fontsize = 25)

plt.show()

ValueError: bad input shape (1609L, 40L)

In [106]:
y_score.shape

(1609L, 40L)

In [ ]:
X_train[0].feature

In [ ]:
res.shape

In [ ]:
y, sr = librosa.load('./wav/id10270/5r0dWxy17C8/00001.wav',sr=None)
mfccs = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=13)

In [ ]:
print mfccs

In [ ]:
np.average(mfccs, 1)

In [ ]:
print mfccs.shape

In [ ]:
mfccs

In [ ]:
y, sr = librosa.load('./wav/id10270/5r0dWxy17C8/00002.wav',sr=None)
mfccs = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=13)

In [ ]:
mfccs